- 윈도우 크롬드라이버 설치 방안
- https://m.blog.naver.com/jsk6824/221763151860

- 맥에서 크롬드라이버 설치 방안
- http://www.epistemology.pe.kr/2018/09/25/1153
- (Homebrew 설치 필요: 터미널에서 

/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install.sh)"

입력)

### 파이썬에서 크롬 브라우저 제어

- 크롬 브라우저 시작

In [59]:
from selenium import webdriver  # selenium 프레임 워크에서 webdriver 가져오기
url = 'http://naver.com'        # 접속할 웹 사이트 주소 (네이버)
driver = webdriver.Chrome('chromedriver')  # 크롬 드라이버로 크롬 켜기, 자기 컴퓨터의 chromedriver 경로로 수정할 것
driver.get(url)                 # 저장한 url 주소로 이동

- 스타벅스 메뉴 가져오기
- https://velog.io/@kho5420/Python-Web-Crawling-Web-Scraping 참조

In [60]:
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver

filename = "data/Starbucks_scraping.csv"
csv_file = open(filename, "w+", encoding="utf-8-sig")
writer = csv.writer(csv_file)

In [64]:
url = "https://www.starbucks.co.kr"
res = webdriver.Chrome('chromedriver') #Chrome('크롬드라이버경로')
res.get(url)
time.sleep(3)  # 3초간 대기
res.find_element_by_xpath("//*[@id='gnb']/div/nav/div/ul/li[2]/h2/a").click()
time.sleep(1) # 1초간 대기
res.find_element_by_xpath("//*[@id='container']/div[6]/div/div[2]/a").click()

In [53]:
html = res.page_source
soup = BeautifulSoup(html, 'html.parser')

items = soup.find_all('li', {"class":"menuDataSet"})

for item in items:
    temp = item.find('img')
    item_img_url = temp['src']
    item_name = temp['alt']
    
    product = (item_name, item_img_url)
    writer.writerow(product)

csv_file.close()

In [54]:
import pandas as pd

star = pd.read_csv("data/Starbucks_scraping.csv", names=['Menu','Image'])
star.head()

,Menu,Image
0,나이트로 바닐라 크림,https://image.istarbucks.co.kr/upload/store/sk...
1,나이트로 쇼콜라 클라우드,https://image.istarbucks.co.kr/upload/store/sk...
2,나이트로 콜드 브루,https://image.istarbucks.co.kr/upload/store/sk...
3,돌체 콜드 브루,https://image.istarbucks.co.kr/upload/store/sk...
4,바닐라 크림 콜드 브루,https://image.istarbucks.co.kr/upload/store/sk...


- 아모레퍼시픽몰 리뷰 수집

In [65]:
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver

filename = "data/amorepacific_scraping.csv"
csv_file = open(filename, "w+", encoding="utf-8-sig")
writer = csv.writer(csv_file)

In [66]:
# 한율 > [리미티드/대용량] 빨간쌀 진액스킨 200ml
url = "https://www.amorepacificmall.com/kr/ko/product/detail?onlineProdSn=39901&onlineProdCode=111650000419"
res = webdriver.Chrome('chromedriver')
res.get(url)
time.sleep(3)  # 3초간 대기
res.find_element_by_xpath("//*[@id='productReview']/button").click()
time.sleep(1) # 1초간 대기
# 처음에 리뷰가 10개만 나옴
# 총 리뷰가 몇 개인지 파악해 모든 리뷰가 나오게 '더보기'를 클릭함
script = res.find_element_by_xpath("//*[@id='ap_container']/div/div[3]/div[2]/div[2]/div[1]/div[1]/div[2]/div/p[2]").text
click_num = int(script.split('명')[0])//10

for i in range(click_num):
    res.find_element_by_xpath("//*[@id='ap_container']/div/div[3]/div[2]/div[2]/div[2]/div[4]/a").click()
    time.sleep(1) # 1초간 대기

In [57]:
html = res.page_source
soup = BeautifulSoup(html, 'html.parser')

items = soup.find_all('div', {"class":"review_box"})

for item in items:
    rating = item.find('span', {'class' : 'sr_only'}).text
    review_rating = rating.split('중 ')[1].split('개')[0]
    review_date = item.find('span', {'class' : 'date'}).text
    review_text = item.find('div', {'class' : 'comment'}).text
    
    product = (review_rating, review_date, review_text)
    writer.writerow(product)

csv_file.close()

In [58]:
import pandas as pd

star = pd.read_csv("data/amorepacific_scraping.csv", names=['Rating', 'Date','Review'])
star.head()

,Rating,Date,Review
0,5,2020.10.22,\n육아 핑계로 화장품 안바른날이 너무 많았고\n한율프로모션 제철도시락 리미트드 사...
1,4,2020.10.21,\n리미티드 대용량이라 하나 더 구매해놨어요 펌핑용기로 계속 나오면 좋겠어요\n더읽...
2,5,2020.10.21,\n한율 빨간쌀 진액 스킨 잘 쓰고 있는데 다 써가서 다시 주문했어요. 대용량으로 ...
3,4,2020.10.21,\n가을되니 기존스킨케어로는 부족해서 건조했는데 빨간쌀 진액스킨을 알아보니 딱일듯 ...
4,5,2020.10.20,\n엄마께 사드렸어요~~촉촉하니 좋다고 하시네요^^\n더읽기\n닫기\n
